In [1]:
from pylab import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import matplotlib.image as mpimg
import os
import urllib
from copy import deepcopy

def grad_descent(f, df, x, t, init_W, alpha):
    """
    the gradient descent function without momentum
    """
    print 'lalala'
    EPS = 1e-5 
    prev_W = init_W-10*EPS
    W = init_W.copy()
    max_iter = 3000
    iter  = 0
    print 'lalala'
    while norm(W - prev_W) >  EPS and iter < max_iter:
        print 'lalala'
        prev_W = W.copy()
        W -= alpha*df(x, t, W)
        if iter % 1 == 0:
            print "Iter", iter
        iter += 1
    return W

def sigmoid(o):
    """
    sigmoid function
    """
    y = 1/(1+np.exp(-o))
    return y

def loss_fn(y, t):
    """
    loss function
    """
    return sum(-t*np.log(y) - (1-t)*np.log(1-y))


def f(x,t,W):
    """
    combined function of forward part
    """
    o = np.dot(x,W.T)
    y = sigmoid(o)
    return loss_fn(y, t)

def df(x,t,W):
    """
    combined function of back propagation part
    """
    o = np.dot(x,W.T)
    y = sigmoid(o)
    lossGradient = y-t
    combined_grad = np.dot(lossGradient.T, x)
    return combined_grad

def performance(trainedW,x,t):
    """
    evaluate the performance of the learning
    """
    o = np.dot(x,trainedW.T)
    y = sigmoid(o)
    corrNum = 0
    for i in range(y.shape[0]):
        if y[i,0] > 0.5:
            y[i,0] = 1
        else:
            y[i,0] = 0
    for i in range(y.shape[0]):
        if y[i,0] == t[i,0]:
            corrNum += 1
    return corrNum / y.shape[0]

training_xy = np.load('training.npy')
X = training_xy[:-1,:].T
t = training_xy[-1, :]

print t.shape
print X.shape
t.reshape([3266,1])
W = np.random.normal(0., 0., (1,5833))
trainedW = grad_descent(f, df, X, t, W, 0.0001)